In [1]:
"""
用于demo一路sEMG&FMG信号进行肌力识别
"""
import pandas as pd

from motiongui import *
from iFEMG_feature import *
from iFEMGprocessing import read_label, pdtable_read_db, data_segment

In [2]:
# read data
raw_data = pd.read_table(r"D:\code\data\iFEMG_strength_level\grade-10.db", sep = ';', header = None)
label = read_label(r"D:\code\data\iFEMG_strength_level\g-10.txt")
# read db file
# row index 0: time
# row index 1-8: FMG signal
# row index 9-16: sEMG signal
# 读取数据array
data_time = raw_data[0].values
raw_FMG = raw_data[7].values
raw_sEMG = raw_data[15].values

FMG = LabeledFMGFeature(raw_FMG, data_time, label, 1223)
FMG.signal_segment_label(0)

In [9]:
sum_len = 0
for i in FMG.active_signal_segment:
    print(len(i))


2456
2439
2488
2481
2427
2456
2441
2465
2436
2513
2451
2478
2449
2452
2460
2473


In [10]:
for i in FMG.rest_signal_segment:
    print(len(i))

2669
3698
3649
3676
3681
3719
3688
3701
3696
3701
3684
3651
3715
3706
3686
3699


In [11]:
 # 读取数据array
data_time = raw_data[0].values
data_FMG = raw_data[7].values
data_sEMG = raw_data[15].values


#将db文件中的时间转换为ms level时间戳
t_stamp = [] # 保存数据文件中时间转换的时间戳，精度ms
for t in data_time:
    t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S,%f")
    ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
    t_stamp.append(ret_stamp)
    
#处理label.txt中的ms level时间戳
# label = read_label("D:\code\data\iFEMG\g-0.txt")
label_t_stamp = [] # 保存label文件中时间对应的时间戳，精度ms
for x in label:
    t = x[0] + " " + x[1]
    t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S.%f")
    ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
    label_t_stamp.append(ret_stamp)
    
# 比较数据文件和label文件中时间戳，划分肌电活动段
# 储存分割好的数据段
sEMG_data_set = []
FMG_data_set = []
rsEMG_data_set = []
rFMG_data_set = []
# 临时一段静息数据和激活数据
temp_sEMG = []
temp_FMG = []
temp_sEMG_r = [] # 静息数据
temp_FMG_r = []

for i in range(len(label) - 1): # 在label时间范围内
    if (label[i][2] == "收缩") and (label[i + 1][2] == "舒张"): # 活动段
        for j in range(len(t_stamp)): # 在整个data长度内搜索，可以优化
            if label_t_stamp[i] <= t_stamp[j] <= label_t_stamp[i + 1]:
                temp_sEMG.append(data_sEMG[j])
                temp_FMG.append(data_FMG[j])
        if len(temp_FMG) != 0:
            sEMG_data_set.append(temp_sEMG)
            FMG_data_set.append(temp_FMG)
        temp_sEMG = []
        temp_FMG = []
    else: # 非活动段，肌肉静息
        for j in range(len(t_stamp)):
            if label_t_stamp[i] <= t_stamp[j] <= label_t_stamp[i + 1]:
                temp_sEMG_r.append(data_sEMG[j])
                temp_FMG_r.append(data_FMG[j])
        rsEMG_data_set.append(temp_sEMG_r)
        rFMG_data_set.append(temp_FMG_r)
        temp_sEMG_r = []
        temp_FMG_r = []

In [12]:
sum_len = 0
for i in FMG_data_set:
    print(len(i))
    sum_len += len(i)


2456
2439
2488
2481
2428
2456
2446
2465
2436
2513
2455
2480
2449
2452
2464
2473


In [13]:
for i in rFMG_data_set:
    print(len(i))
    sum_len += len(i)

print(sum_len)
print(raw_FMG.shape)

2456
2439
2488
2481
2428
2456
2446
2465
2436
2513
2455
2480
2449
2452
2464
2473
78762
(190667,)
